In [1]:
!pip install scrapy -q
    
import pandas as pd

In [9]:
df = pd.read_csv('Final_file.csv')

In [10]:
cities = df["Cities"].unique()

cities = [c.replace(' ', '+') for c in cities]

cities_url = []
for i in cities:
    cities_url.append('https://www.booking.com/searchresults.fr.ss='+i)

In [11]:
# Import os => Library used to easily manipulate operating systems
## More info => https://docs.python.org/3/library/os.html
import os 

# Import logging => Library used for logs manipulation 
## More info => https://docs.python.org/3/library/logging.html
import logging

# Import scrapy and scrapy.crawler 
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.http import Request

In [12]:
class BookingSpider(scrapy.Spider):
    # Name of your spider
    name = "Booking"

    # Starting URL
    
    start_urls = cities_url
    def parse(self, response):
        results = response.css('div.sr_item')
        
        
        for h in results:
            City = cities
            yield {
                'City' : h.css('a.bui-link::text').get(),
                'Hotel name': h.css('span.sr-hotel__name::text').get(),
                'URL': ('https://www.booking.com') + (h.css('a').attrib["href"]),
                'Coordonnées latlong': h.css('a::attr(data-coords)').get(),
                'Description': h.css('div.hotel_desc::text').get(),
                'Note de satisfaction': h.css('div.bui-review-score__badge::text').get(),
                'Ville' : response.request.url

                }
        
"""
        # Scraper plusieurs pages par ville (consomme trop de ressources -> JSON de 2,7GB obtenu)
        # Select the NEXT button and store it in next_page
            try:
                next_page = response.css('a.bui-pagination__link').attrib["href"]
                print(i)
            except KeyError:
                logging.info('No next page. Terminating crawling process.')
            else:
                yield response.follow(next_page, callback=self.parse)
"""

'\n        # Scraper plusieurs pages par ville (consomme trop de ressources -> JSON de 2,7GB obtenu)\n        # Select the NEXT button and store it in next_page\n            try:\n                next_page = response.css(\'a.bui-pagination__link\').attrib["href"]\n                print(i)\n            except KeyError:\n                logging.info(\'No next page. Terminating crawling process.\')\n            else:\n                yield response.follow(next_page, callback=self.parse)\n'

In [13]:
# Name of the file where the results will be saved
filename = "liste_hotels.json"

# If file already exists, delete it before crawling (because Scrapy will concatenate the last and new results otherwise)
if filename in os.listdir('results/'):
        os.remove('results/' + filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)',
    'LOG_LEVEL': logging.INFO,
    'AUTOTHROTTLE_ENABLED': True,
    'AUTOTHROTTLE_TARGET_CONCURRENCY' : 0.1,
    "FEEDS": {
        'results/' + filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(BookingSpider)
process.start()

2021-12-02 16:06:18 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-12-02 16:06:18 [scrapy.utils.log] INFO: Versions: lxml 4.6.4.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Linux-5.4.129+-x86_64-with-glibc2.10
2021-12-02 16:06:18 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'AUTOTHROTTLE_TARGET_CONCURRENCY': 0.1,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko)'}
2021-12-02 16:06:18 [scrapy.extensions.telnet] INFO: Telnet Password: c3ab4f1946152f66
2021-12-02 16:06:18 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensi

In [15]:
#Transformer le JSON en pandas Dataframe

df = pd.read_json('liste_hotels5.json')
df.head()

,City,Hotel name,URL,Coordonnées latlong,Description,Note de satisfaction,Ville
0,\nLe Mont Saint Michel\n,\nHôtel Vert\n,https://www.booking.com/hotel/fr/vert.en-gb.ht...,"-1.50961697101593,48.6147004862904",\nHotel Vert offers pastel-coloured rooms with...,8.1,https://www.booking.com/searchresults.html?si=...
1,\nLe Mont Saint Michel\n,\nMercure Mont Saint Michel\n,https://www.booking.com/hotel/fr/mont-saint-mi...,"-1.51054501533508,48.6142465295929",\nThis Mercure is situated in parkland just 2 ...,8.2,https://www.booking.com/searchresults.html?si=...
2,\nLe Mont Saint Michel\n,\nHotel De La Digue\n,https://www.booking.com/hotel/fr/de-la-digue.e...,"-1.51091784238815,48.6168815494412",\nThe traditional Hotel De La Digue in the Bas...,7.1,https://www.booking.com/searchresults.html?si=...
3,\nLe Mont Saint Michel\n,\nLe Saint Aubert\n,https://www.booking.com/hotel/fr/hotel-saint-a...,"-1.51010513305664,48.6129378347065","\nSet amid lush greenery, just 2 km from the M...",7.3,https://www.booking.com/searchresults.html?si=...
4,\nLe Mont Saint Michel\n,\nHotel Gabriel\n,https://www.booking.com/hotel/fr/hotel-gabriel...,"-1.5107099711895,48.6153814136834",\nHotel Gabriel is located 1.6 Km from Mont Sa...,8.1,https://www.booking.com/searchresults.html?si=...


In [85]:
"""#Suppression des caractères inutiles

df['Hotel name'] = df['Hotel name'].str.replace('\n' , '')
df['Description'] = df['Description'].str.replace('\n' , '')
df['City'] = df['City'].str.replace('\n' , '')
df['Ville'] = df['Ville'].str.replace("\%","").str.replace("=","").str.replace(";","").str.replace("?","").str.replace("/","")
df['Ville'] = df['Ville'].str.replace(";ifl=1;aid=304142;label=gen173nr-1BCBcoggI46AdIM1gEaBWIAQGYAQm4ARjIAQXYAQHoAQGIAgGoAgS4Av-BxooGwAIB0gIkOTgxN2QzYWMtNWNhZC00ODA0LTgzMWYtMDY4ZmZlZjgxMTIy2AIF4AIB", '')
df['Ville'] = df['Ville'].str.replace("https:www.booking.comsearchresults.htmlsiai2Cco2Cci2Cre2Cla2Cdisssearchresults.fr.ss","").str.replace("+"," ")
df['Ville'] = df['Ville'].str.replace("ifl1aid304142labelgen173nr-1BCBcoggI46AdIM1gEaBWIAQGYAQm4ARjIAQXYAQHoAQGIAgGoAgS4ApiCxooGwAIB0gIkODIxZGJmYTgtMTkyYi00YzU3LWIzNTUtNzBhZTRjMDRiZWQy2AIF4AIB","").str.replace("+"," ")

df.head(5)"""

,City,Hotel name,URL,Coordonnées latlong,Description,Note de satisfaction,Ville
0,Le Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,"-1.50961697101593,48.6147004862904",Hotel Vert offers pastel-coloured rooms with a...,8.1,Le Mont Saint Michel
1,Le Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"-1.51054501533508,48.6142465295929",This Mercure is situated in parkland just 2 km...,8.2,Le Mont Saint Michel
2,Le Mont Saint Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.e...,"-1.51091784238815,48.6168815494412",The traditional Hotel De La Digue in the Basse...,7.1,Le Mont Saint Michel
3,Le Mont Saint Michel,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,"-1.51010513305664,48.6129378347065","Set amid lush greenery, just 2 km from the Mon...",7.3,Le Mont Saint Michel
4,Le Mont Saint Michel,Hotel Gabriel,https://www.booking.com/hotel/fr/hotel-gabriel...,"-1.5107099711895,48.6153814136834",Hotel Gabriel is located 1.6 Km from Mont Sain...,8.1,Le Mont Saint Michel


In [8]:
#Formatage et cleaning des données

df['Hotel name'] = df['Hotel name'].str.replace('\n' , '')
df['Description'] = df['Description'].str.replace('\n' , '')
df['City'] = df['City'].str.replace('\n' , '')
df['Ville'] = df['Ville'].str.replace("\%","").str.replace("=","").str.replace(";","").str.replace("?","").str.replace("/","")
df['Ville'] = df['Ville'].str.replace("https:www.booking.comsearchresults.htmlsiai2Cco2Cci2Cre2Cla2Cdisssearchresults.fr.ss","")

df.tail(5)

,City,Hotel name,URL,Coordonnées latlong,Description,Note de satisfaction,Ville
728,"La Rochelle City Centre, La Rochelle",Hotel de Paris,https://www.booking.com/hotel/fr/de-paris-la-r...,"-1.15117460489273,46.1619691141879",The Hotel de Paris is located in the city cent...,7.9,La+Rochelleifl1aid304142labelgen173nr-1DCBcogg...
729,"La Rochelle City Centre, La Rochelle",Hotel de La Paix,https://www.booking.com/hotel/fr/de-la-paix-la...,"-1.15085542169254,46.161939390064",Located in the heart of the city and footsteps...,8.0,La+Rochelleifl1aid304142labelgen173nr-1DCBcogg...
730,"La Rochelle City Centre, La Rochelle",Hôtel Le Rupella,https://www.booking.com/hotel/fr/le-rupella.en...,"-1.15106999874115,46.1581749442479","Located in the heart of La Rochelle, just a 10...",7.2,La+Rochelleifl1aid304142labelgen173nr-1DCBcogg...
731,"La Rochelle City Centre, La Rochelle",Best Western Premier Masqhotel,https://www.booking.com/hotel/fr/masqhotel.en-...,"-1.14505648612976,46.1548367218857",The Best Western Masqhotel is a design hotel l...,8.6,La+Rochelleifl1aid304142labelgen173nr-1DCBcogg...
732,"La Rochelle City Centre, La Rochelle",Hôtel Le Yachtman,https://www.booking.com/hotel/fr/le-yachtman.e...,"-1.14996761083603,46.1572836509306","Located in La Rochelle, Le Yachtman features a...",7.6,La+Rochelleifl1aid304142labelgen173nr-1DCBcogg...


In [9]:
#Remplacer le lien de départ par le nom de la ville dans l'URL

df['Ville'] = df['Ville'].str.replace("ifl1aid"," ")
df[['Ville','Ville2']] = df["Ville"].str.split(' ', expand=True)

df = df.drop(['Ville2'] , axis=1)

df['Ville'] = df['Ville'].str.replace("+"," ")

In [10]:
df.head()

,City,Hotel name,URL,Coordonnées latlong,Description,Note de satisfaction,Ville
0,Le Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,"-1.50961697101593,48.6147004862904",Hotel Vert offers pastel-coloured rooms with a...,8.1,Le Mont Saint Michel
1,Le Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"-1.51054501533508,48.6142465295929",This Mercure is situated in parkland just 2 km...,8.2,Le Mont Saint Michel
2,Le Mont Saint Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.e...,"-1.51091784238815,48.6168815494412",The traditional Hotel De La Digue in the Basse...,7.1,Le Mont Saint Michel
3,Le Mont Saint Michel,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,"-1.51010513305664,48.6129378347065","Set amid lush greenery, just 2 km from the Mon...",7.3,Le Mont Saint Michel
4,Le Mont Saint Michel,Hotel Gabriel,https://www.booking.com/hotel/fr/hotel-gabriel...,"-1.5107099711895,48.6153814136834",Hotel Gabriel is located 1.6 Km from Mont Sain...,8.1,Le Mont Saint Michel


In [34]:
#Renommer colonne pour spliter ensuite sur les virgules

df = df.rename(columns={'Coordonnées latlong': 'Latitude,Longitude'})
df.head()

,City,Hotel name,URL,"Latitude,Longitude",Description,Note de satisfaction,Ville
0,Le Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,"-1.50961697101593,48.6147004862904",Hotel Vert offers pastel-coloured rooms with a...,8.1,Le Mont Saint Michel
1,Le Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"-1.51054501533508,48.6142465295929",This Mercure is situated in parkland just 2 km...,8.2,Le Mont Saint Michel
2,Le Mont Saint Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.e...,"-1.51091784238815,48.6168815494412",The traditional Hotel De La Digue in the Basse...,7.1,Le Mont Saint Michel
3,Le Mont Saint Michel,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,"-1.51010513305664,48.6129378347065","Set amid lush greenery, just 2 km from the Mon...",7.3,Le Mont Saint Michel
4,Le Mont Saint Michel,Hotel Gabriel,https://www.booking.com/hotel/fr/hotel-gabriel...,"-1.5107099711895,48.6153814136834",Hotel Gabriel is located 1.6 Km from Mont Sain...,8.1,Le Mont Saint Michel


In [35]:
#Splitter latitude et longitude

df[['latitude','longitude']] = df["Latitude,Longitude"].str.split(',', expand=True)
df = df.drop(['Latitude,Longitude'], axis=1)
df.head()

,City,Hotel name,URL,Description,Note de satisfaction,Ville,latitude,longitude
0,Le Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,Hotel Vert offers pastel-coloured rooms with a...,8.1,Le Mont Saint Michel,-1.50961697101593,48.6147004862904
1,Le Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,This Mercure is situated in parkland just 2 km...,8.2,Le Mont Saint Michel,-1.51054501533508,48.6142465295929
2,Le Mont Saint Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.e...,The traditional Hotel De La Digue in the Basse...,7.1,Le Mont Saint Michel,-1.51091784238815,48.6168815494412
3,Le Mont Saint Michel,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,"Set amid lush greenery, just 2 km from the Mon...",7.3,Le Mont Saint Michel,-1.51010513305664,48.6129378347065
4,Le Mont Saint Michel,Hotel Gabriel,https://www.booking.com/hotel/fr/hotel-gabriel...,Hotel Gabriel is located 1.6 Km from Mont Sain...,8.1,Le Mont Saint Michel,-1.5107099711895,48.6153814136834


In [36]:
df.to_csv('Hotel_Data_Clean.csv')

In [37]:
!pip install plotly

  Using cached plotly-5.3.1-py2.py3-none-any.whl (23.9 MB)
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)


In [38]:
df[['latitude','longitude','Note de satisfaction']] = df[['latitude','longitude','Note de satisfaction']].astype(float)

In [146]:
#df.to_csv('Export Test.csv')

In [11]:
df2 = pd.read_csv('Final_file.csv')

In [68]:
#Renommer certaines colonnes pour le merging

df2 = df2.rename(columns={'Cities': 'Ville'})
df2 = df2.rename(columns={'latitude': 'latitude ville'})
df2 = df2.rename(columns={'longitude': 'longitude ville'})

df2.head()

,Unnamed: 0,Index,Ville,latitude ville,longitude ville,Jour,Degrés,Taux
0,0,0,Mont Saint Michel,48.635523,-1.510257,j1,20.22,51.0
1,1,1,St Malo,48.649518,-2.026041,j1,17.89,66.0
2,2,2,Bayeux,49.276462,-0.702474,j1,16.93,56.0
3,3,3,Le Havre,49.493898,0.107973,j1,15.86,59.0
4,4,4,Rouen,49.440459,1.093966,j1,17.94,49.0


In [69]:
df['Ville'] = df['Ville'].astype(str)
df2['Ville'] = df2['Ville'].astype(str)

In [70]:
df_combined = df.merge(df2, on='Ville')

df_combined.head()

,City,Hotel name,URL,Description,Note de satisfaction,Ville,latitude,longitude,Unnamed: 0,Index,latitude ville,longitude ville,Jour,Degrés,Taux
0,"Sillon, Saint Malo","Appartement Saint-Malo, 2 pièces, 4 personnes ...",https://www.booking.com/hotel/fr/gite-saint-ma...,"Appartement Saint-Malo, 2 pièces, 4 personnes ...",NaN,St Malo,-2.016065,48.652266,1,1,48.649518,-2.026041,j1,17.89,66.0
1,"Sillon, Saint Malo","Appartement Saint-Malo, 2 pièces, 4 personnes ...",https://www.booking.com/hotel/fr/gite-saint-ma...,"Appartement Saint-Malo, 2 pièces, 4 personnes ...",NaN,St Malo,-2.016065,48.652266,36,36,48.649518,-2.026041,j2,17.24,58.0
2,"Sillon, Saint Malo","Appartement Saint-Malo, 2 pièces, 4 personnes ...",https://www.booking.com/hotel/fr/gite-saint-ma...,"Appartement Saint-Malo, 2 pièces, 4 personnes ...",NaN,St Malo,-2.016065,48.652266,71,71,48.649518,-2.026041,j3,16.97,71.0
3,"Sillon, Saint Malo","Appartement Saint-Malo, 2 pièces, 4 personnes ...",https://www.booking.com/hotel/fr/gite-saint-ma...,"Appartement Saint-Malo, 2 pièces, 4 personnes ...",NaN,St Malo,-2.016065,48.652266,106,106,48.649518,-2.026041,j4,18.25,74.0
4,"Sillon, Saint Malo","Appartement Saint-Malo, 2 pièces, 4 personnes ...",https://www.booking.com/hotel/fr/gite-saint-ma...,"Appartement Saint-Malo, 2 pièces, 4 personnes ...",NaN,St Malo,-2.016065,48.652266,141,141,48.649518,-2.026041,j5,19.48,60.0


In [93]:
#Best Hotels by color (note de satisfaction > 8.5)

import plotly.express as px
import matplotlib.pyplot as plt
import plotly.io as pio
pio.renderers.default="iframe_connected"

mask = (df['Note de satisfaction'] > 8.5) 
data_viz = df.loc[mask,:]

fig = px.scatter_mapbox(data_viz, lat="longitude", lon="latitude", color="Note de satisfaction", mapbox_style="carto-positron", zoom=4, hover_name='Hotel name', size='Note de satisfaction', size_max=8)
fig.show()

In [94]:
#Export dataset en csv
df_combined.to_csv('File_to_load_S3.csv')